In [1]:
import numpy as np
import matplotlib.pyplot as plt

from CADMium import Kohnsham
from CADMium import Pssolver
from CADMium import Psgrid
from CADMium import Partition
from CADMium import Inverter

%matplotlib widget

---
### Perform Isolated Li atom Calculation. 
Code should run as it is but for idential calculations increase to grid size to: [7,12,12]


In [58]:
#Distance of the nucley from grid center
a =  5.122/2

#Nuclear charges on centers AB
Za  = 3
Zb = 0

#Set polaization. 1 Unpolarized, 2 Polarized
pol = 1

Nmo = [[2]]
N   = [[3]]

optKS = {
        "interaction_type" : "dft",
        "SYM" : False,
        "FRACTIONAL" : True,
        }

#Grid Options
NP = 7 #Number of points per block
NM =  [4,4] #Number of blocks [angular, radial]
L = np.arccosh(15./a) #Maximum radial coordinate value
loc = np.array(range(-4,5)) #Non inclusive on upper bound

#Create and initialize grid object
grid = Psgrid(NP, NM, a, L, loc)
grid.initialize()

#Kohn Sham object
KS = Kohnsham(grid, Za, Zb, pol, Nmo, N, optKS)
KS.scf({})

#Store information of KS
# KS0 = KS.copy()

#Extract Isolated Fragment Density. 
Isolated_D = KS.n.copy()

---
### Perform PDFT Calculation. 
Currently the method used is "OrbitalInvert". 
But original code may have used "WuYang". 
Code should run as it is but for idential calculations increase to grid size to: [7,12,12]

In [47]:
a = 5.122/2
#Nuclear charge for fragments A and B
Za, Zb = 3,3
#Set polarization 1-Unpolarized, 2-Polarized
pol = 2
#Fragment a electrons [alpha, beta]
Nmo_a = [[2,1]] #Number of molecular orbitals to calculate
N_a   = [[2,1]]
#Ensemble mix
nu_a = 1
#Fragment b electrons
Nmo_b = [[2,1]]
N_b   = [[2,1]]
#Ensemble mix
nu_b = 1

#Molecular elctron configuration
Nmo_m = [[3,3]]
N_m   = [[3,3]]


#Initialize required objects. And make calculation in isolated fragments for initial guess. 

part = Partition(grid, Za, Zb, pol, Nmo_a, N_a, nu_a, Nmo_b, N_b, nu_b, { "kinetic_part_type" : 'inversion',
                                                                          "ab_sym"            : True,
                                                                          "ens_spin_sym"      : True,})

#Setup inverter object
mol_solver = Pssolver(grid, Nmo_m, N_m)
part.inverter = Inverter(grid, mol_solver, {"invert_type"     : "orbitalinvert",
                                            "tol_invert"      : 1e-10,
                                            "max_iter_invert" : 40,
                                            "disp"            : False,
                                            "ab_sym"          : True,
                                            "ens_spin_sym"    : True,})

part.optPartition.isolated = True
part.scf({"disp"  : True,
          "alpha" : [0.6],
          "e_tol" : 1e-12})


#Turn off iterative linear solver for each solver
part.KSa.solver[0][0].optSolver.iter_lin_solver = False
part.KSa.solver[0][1].optSolver.iter_lin_solver = False


part.optPartition.isolated   = False

part.scf({"disp"       : True,
          "alpha"      : [0.6],
          "max_iter"   : 50,
          "e_tol"      : 1e-10,
          "continuing" : True})

#Store full densities under the presence of vp.
Da_vp = part.nf[:,0].copy()
Db_vp = part.nf[:,1].copy()

---
### Perform Isolated Li Calculation under the presence of Vp. 
You are required to use the Kohn Sham calculation used in the original isolated fragment calculation. 
Code should run as it is but for idential calculations increase to grid size to: [7,12,12]

In [49]:
#Store vp from pdft calculation. 
vp = part.V.vp.copy()
vp = vp[:,0] + vp[:,1] 


#Setting continue : True is imoprtant otherwise veff gets scratched. 
KS.solver[0][0].veff += vp
KS.solver[0][0].calc_orbitals()
KS.solver[0][0].calc_density()
KS.solver[0][0].calc_energy()

#Save density under the presence of vp
vp_D = KS.n.copy()

---
### Generate Figure 9. Parititon Potential. 

In [51]:
full, x,y = grid.plotter(part.V.vp[:,0] + part.V.vp[:,1])
fig = plt.figure(dpi=150)
plt.contourf(x,y,full, levels=50, cmap="jet")
plt.xlim([-5,5])
plt.ylim([-5,5])
plt.colorbar()
# plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

---
### Generate Figure 9. Difference between Fragment Density and Isolated Atomic Density. 

In [57]:
D_grid, x, y = grid.plotter(Isolated_D)
D_vp_grid, _, _ = grid.plotter(Da_vp)

fig = plt.figure(dpi=150)
plt.contourf(x,y, D_grid - D_vp_grid, levels=100, cmap="jet")
plt.xlim([-5,5])
plt.ylim([-5,5])
plt.colorbar()
# plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

---
### Generate Figure 11. Components of the Partition Potential

In [43]:
x_axis, vp = grid.axis_plot(part.V.vp[:,0] + part.V.vp[:,1])
x_axis, vp_kin = grid.axis_plot(part.V.vp_kin[:,0] + part.V.vp_kin[:,1])
x_axis, vp_xc = grid.axis_plot(part.V.vp_x[:,0] + part.V.vp_x[:,1] + part.V.vp_c[:,0] + part.V.vp_c[:,1])
x_axis, vp_hext = grid.axis_plot( part.V.vp_h[:,0] + part.V.vp_h[:,1] + part.V.vp_pot[:,0] + part.V.vp_pot[:,1])

fig = plt.figure(dpi=150)
plt.plot(x_axis, vp, label='Total')
plt.plot(x_axis, vp_kin, label='Kinetic')
plt.plot(x_axis, vp_xc, label='XC')
plt.plot(x_axis, vp_hext, label="H + Vext")

plt.xlim(0,7)
plt.ylim(-0.2, 0.2)

plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

---
### Generate Table 9. Energies and Components of Ep, in atomic Units

In [68]:
values = {}
for i in part.E.__dict__:
    if i.startswith("__") is False:
        values.update({i : getattr(part.E, i)})
values

{'Ea': -7.337784850223082,
 'Eb': -7.337784850223082,
 'Ef': -14.675569700446164,
 'Tsf': 14.515977605630539,
 'Eksf': array([[-3.87274135, -3.62105117]]),
 'Enucf': -33.88821469685777,
 'Exf': -3.0418434866746957,
 'Ecf': -0.30099393923990164,
 'Ehf': 8.039504816695667,
 'Vhxcf': 11.684634295466644,
 'Ep': -1.8069182207761783,
 'Ep_pot': -3.6782118547644824,
 'Ep_kin': 0.0047843397213842564,
 'Ep_hxc': 1.86650929426692,
 'Et': -16.482487921222344,
 'Vnn': 1.757126122608356,
 'E': -14.725361798613987,
 'evals_a': array([], dtype=float64),
 'evals_b': array([], dtype=float64),
 'Ep_h': 1.8866106227315615,
 'Ep_x': 0.007092234668995001,
 'Ep_c': -0.027193563133636578}

In [ ]:
x_axis,vp = grid.axis_plot(part.V.vp[:,0] + part.V.vp[:,1])